# **2. Baseline**
In the first place, we have tried to define a baseline to better understand the dataset potentiality and which value of MAPE we can expect to achive. <br>
For the baseline we have tried to use the sales of the previous week as our prediction.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.append("../")

csv_train = pd.read_csv("../dataset/original/train.csv")
csv_test = pd.read_csv("../dataset/original/x_test.csv")

In [2]:
from preprocessing.preprocessing import preprocessing, train_validation_split
df = preprocessing(csv_train, csv_test, useTest=False)
df_t = df[~df.target.isna()]
df_train, df_val, val_dates = train_validation_split(df_t, k=0.10)
df_val

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target
2942,2019-02-23,1603,1,175.74,2,0.50,0.0,0.650616,11.407654,0,11.392756,88676.0
3474,2019-02-23,1732,0,87.87,2,1.25,0.0,0.208052,9.315061,0,9.301186,10950.0
5469,2019-02-23,2705,1,131.80,0,0.56,0.0,0.699400,11.222680,0,11.222867,74821.0
5336,2019-02-23,2704,1,878.68,0,2.75,0.0,0.422486,9.912992,0,9.914774,20226.0
4538,2019-02-23,2681,0,105.44,0,1.64,0.0,89.460784,8.409385,0,8.383662,4374.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4289,2019-06-22,2410,1,395.41,4,1.72,3.0,100.000000,9.423838,0,9.030855,8356.0
2693,2019-06-22,1554,1,395.41,2,1.00,0.0,0.000000,10.767116,0,10.724654,45462.0
3624,2019-06-22,2249,1,395.41,4,1.73,3.0,93.796134,9.339701,0,8.936956,7607.0
1712,2019-06-22,1065,1,125.65,3,0.56,3.0,36.370096,11.621457,1,11.554048,104197.0


In [3]:
df_val = df_val.sort_values(['sku', 'Date'])
predictions = []

for s in sorted(set(df_val.sku)):
    current_sku_pred = []
    current_sku_pred.append(df_train[df_train.sku == s]['real_target'].tail(1).values.squeeze())    
    current_sku_pred.extend(df_val[df_val.sku == s]['real_target'].shift(1)[1:])
    predictions.extend(current_sku_pred)

df_val['predictions'] = predictions
df_val

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target,predictions
115,2019-02-23,144,0,114.23,1,3.49,1.0,4.215402,9.812632,1,9.502039,13386.0,18262.0
116,2019-03-02,144,0,114.23,1,3.48,0.0,0.000000,9.502039,1,9.625558,15146.0,13386
117,2019-03-09,144,0,114.23,1,2.17,0.0,0.000000,9.625558,1,10.948277,56855.0,15146
118,2019-03-16,144,0,114.23,1,2.00,40.0,99.105795,10.948277,1,11.368547,86555.0,56855
119,2019-03-23,144,0,114.23,1,2.00,48.0,98.821982,11.368547,1,11.329687,83256.0,86555
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,2019-05-25,2718,1,395.41,0,1.11,0.0,26.050480,10.430462,0,10.414183,33328.0,33875
6015,2019-06-01,2718,1,395.41,0,1.30,1.0,43.099496,10.414183,0,10.021848,22512.0,33328
6016,2019-06-08,2718,1,395.41,0,1.55,0.0,0.000000,10.021848,0,9.767782,17461.0,22512
6017,2019-06-15,2718,1,395.41,0,1.55,0.0,0.000000,9.767782,0,9.747185,17105.0,17461


In [4]:
from metrics.MAPE import MAPE
print("Baseline validation MAPE: %f" % MAPE(df_val['real_target'], df_val['predictions']))

Baseline validation MAPE: 16.313507


This is the result on the validation of our baseline, a very simple model which can be surely improved.